# 환경설정

In [3]:
import pandas as pd 
import os

df = pd.read_json(os.path.join("..","..","data","TA_User_Reviws_Korea_all.json"))
df.head()

,id,language,reviewStatus,title,helpfulVotes,createdDate,publishedDate,rating,text,photoIds,...,location.thumbnail,route,location.route,userProfile.route,location.reviewSummary,location.parent,userProfile.hometown.location.parent,userProfile.contributionCounts,location,userProfile.avatar
0,748454819,en,PUBLISHED,Great Hotel,0,2020-03-01,2020-03-01,5,I really enjoyed our stay here in Gracery Hote...,[453807126],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,744980685,en,PUBLISHED,Vibrant and Trendy,0,2020-02-14,2020-02-13,5,This is a place for you to laidback and have s...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,744978150,en,PUBLISHED,So Yummy!,0,2020-02-14,2020-02-13,4,The chicken was so delicious. its crisp and cr...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,744977340,en,PUBLISHED,Super Luv,0,2020-02-14,2020-02-13,5,"First, the location. Its located just right be...",[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,744973034,en,PUBLISHED,Chaotic Street Food,0,2020-02-14,2020-02-13,4,This is the place where you can shop any cosme...,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 1. FM

In [ ]:
# ! pip install git+https://github.com/coreylynch/pyFM

In [4]:
df = df.loc[:,['userId', 'location.locationId', 'rating']]
df.head() 

,userId,location.locationId,rating
0,F9AD0C1F6409A1574AD0AFC237D2A4CE,14159911,5
1,23EF5413495FF530151DB7E5C0FEF38D,1958940,5
2,23EF5413495FF530151DB7E5C0FEF38D,5979642,4
3,23EF5413495FF530151DB7E5C0FEF38D,13426084,5
4,23EF5413495FF530151DB7E5C0FEF38D,553546,4


In [ ]:
dff = df.groupby('userId')['userId'].agg([('user_count','count')]).reset_index()
one_user = dff.loc[dff['user_count']==1]['userId']

df_one = df.loc[df.userId.isin(one_user.values)]
df_more = df.loc[~df.userId.isin(one_user.values)]

from sklearn.model_selection import train_test_split
base, test = train_test_split(df_more, 
                                   stratify=df_more['userId'], 
                                   test_size=0.20, 
                                   random_state=42) 

print('# interactions on base set: %d' % len(base))
print('# interactions on test set: %d' % len(test))
print('\n')

base = pd.concat([base, df_one])

print('# interactions on base set: %d' % len(base))
print('# interactions on test set: %d' % len(test))

# base.to_csv('/content/gdrive/My Drive/tobigs_recommend/Phase 3. Collaborative Modeling/유나_try/Matrix_Completion/base.txt', index=False, header=None, sep="\t")
# test.to_csv('/content/gdrive/My Drive/tobigs_recommend/Phase 3. Collaborative Modeling/유나_try/Matrix_Completion/test.txt', index=False, header=None, sep="\t")

# interactions on base set: 26488
# interactions on test set: 6622


# interactions on base set: 30575
# interactions on test set: 6622


In [ ]:
print(base.shape, test.shape)

(30575, 3) (6622, 3)


https://github.com/coreylynch/pyFM

In [ ]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm

# Read in data
def loadData(filename,path="/content/gdrive/My Drive/tobigs_recommend/Phase 3. Collaborative Modeling/유나_try/Matrix_Completion/"):
    data = []
    y = []
    users=set()
    items=set()
    with open(path+filename) as f:
        for line in f:
            (userId,locationId,rating)=line.split('\t')
            data.append({ "user_id": str(userId), "location_id": str(locationId)})
            y.append(float(rating))
            users.add(userId)
            items.add(locationId)

    return (data, np.array(y), users, items)

(train_data, y_train, train_users, train_items) = loadData("base.txt")
(test_data, y_test, test_users, test_items) = loadData("test.txt")
v = DictVectorizer() 
X_train = v.fit_transform(train_data) 
X_test = v.transform(test_data)

In [ ]:
print(train_data)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from pyfm import pylibfm
# Build and train a Factorization Machine
fm = pylibfm.FM(num_factors=10, num_iter=100, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")

fm.fit(X_train,y_train) 

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.45983
-- Epoch 2
Training MSE: 0.37282
-- Epoch 3
Training MSE: 0.36292
-- Epoch 4
Training MSE: 0.35605
-- Epoch 5
Training MSE: 0.35045
-- Epoch 6
Training MSE: 0.34558
-- Epoch 7
Training MSE: 0.34118
-- Epoch 8
Training MSE: 0.33722
-- Epoch 9
Training MSE: 0.33377
-- Epoch 10
Training MSE: 0.33028
-- Epoch 11
Training MSE: 0.32719
-- Epoch 12
Training MSE: 0.32425
-- Epoch 13
Training MSE: 0.32136
-- Epoch 14
Training MSE: 0.31874
-- Epoch 15
Training MSE: 0.31614
-- Epoch 16
Training MSE: 0.31367
-- Epoch 17
Training MSE: 0.31117
-- Epoch 18
Training MSE: 0.30893
-- Epoch 19
Training MSE: 0.30674
-- Epoch 20
Training MSE: 0.30453
-- Epoch 21
Training MSE: 0.30244
-- Epoch 22
Training MSE: 0.30030
-- Epoch 23
Training MSE: 0.29837
-- Epoch 24
Training MSE: 0.29642
-- Epoch 25
Training MSE: 0.29449
-- Epoch 26
Training MSE: 0.29250
-- Epoch 27
Training MSE: 0.29062
-- Epoch 28
Tra

In [ ]:
# Evaluate 
preds = fm.predict(X_test) 
from sklearn.metrics import mean_squared_error
print("FM MSE: %.4f" % mean_squared_error(y_test,preds))

FM MSE: 0.6446


In [ ]:
for i in range(len(preds)):
  test_data[i]['preds'] = preds[i]

preds_set = pd.DataFrame(test_data)

,user_id,movie_id,preds
0,1ADA2A82D2C08F1F68ADAE0906209893,12635586,4.702952
1,DB62DED2351FD52490A7CE1C6E7C3832,301817,3.402326
2,80347C94049BCA63983E6DBDBAD9A17F,7747972,3.731286
3,D0FFFA733EECD4672EF671078332C2EE,1507786,3.013502
4,45AE62F774935F2028E98DB1151AB486,7064253,3.992357
...,...,...,...
6617,DE6714A7E58640C645814068F15E938D,1958940,3.444039
6618,D904E3DA6A57C8A019040BD5E9B70418,12369399,3.862486
6619,D12206BF49AF7716E00AF0C27E95C071,7613319,4.662172
6620,24405C44B0C5098B474ADDADCBA85F3D,7899601,4.526094


# 2. 전체 데이터로 학습 후 빈공간 생성 후 예측 (FM 직접구현 sibar....;) 


In [4]:
df = df.loc[:,['userId', 'location.locationId', 'rating']]
df.head()

,userId,location.locationId,rating
0,F9AD0C1F6409A1574AD0AFC237D2A4CE,14159911,5
1,23EF5413495FF530151DB7E5C0FEF38D,1958940,5
2,23EF5413495FF530151DB7E5C0FEF38D,5979642,4
3,23EF5413495FF530151DB7E5C0FEF38D,13426084,5
4,23EF5413495FF530151DB7E5C0FEF38D,553546,4


In [5]:
# 유저-아이템 매트릭스 값들을 데이터 프레임 행 펼침 
df_pivot = pd.pivot_table(df, index='userId', columns='location.locationId', values='rating').fillna(0)
df_pivot = df_pivot.reset_index()

new_df = pd.melt(df_pivot, id_vars='userId')
new_df.rename(columns={'value':'rating'}, inplace=True)
new_df 

,userId,location.locationId,rating
0,0009720721B493BAE608AA592B9CE7AE,294197,0.0
1,00150C8ACD87B7A3A54497D923CB3358,294197,0.0
2,001608AFA00EC79804E14EE201018BBB,294197,0.0
3,00178BFBBC5B052A9E0845691F9585B4,294197,0.0
4,001A6C668BB5BE75A85603BBADAEBCD9,294197,0.0
...,...,...,...
82936017,FFD763E06A0001B0F8143C91D2EE35FB,20903295,0.0
82936018,FFF028A54917977AB9549FD75103ECDA,20903295,0.0
82936019,FFF66251C5DC520DA1F9D217419BD90A,20903295,0.0
82936020,FFFA706118AC068F63A24BD7452D4F80,20903295,0.0


In [6]:
# userId int화 
dff = new_df.groupby('userId')['userId'].agg([('user','count')]).reset_index()
dff = dff.reset_index()
dff.rename(columns={'index':'userID'},inplace=True)
dff = dff.loc[:,['userId','userID']]

# 원본 userid 정보 연결
users = dff.drop_duplicates(['userId'], keep='first')
users.to_csv('users.csv', index=False)

new_dff = pd.merge(new_df, dff, on='userId')
new_dff = new_dff.iloc[:,1:]
new_dff['location.locationId'] = new_dff['location.locationId'].astype('int')
new_dff

,location.locationId,rating,userID
0,294197,0.0,0
1,299151,0.0,0
2,299152,0.0,0
3,299154,0.0,0
4,299502,0.0,0
...,...,...,...
82936017,20403351,0.0,7612
82936018,20483591,0.0,7612
82936019,20494843,0.0,7612
82936020,20860800,0.0,7612


In [10]:
test = new_dff.query('rating == 0.0')
train = new_dff.query('rating != 0.0')
print(test.shape, train.shape)

(82900038, 3) (35984, 3)


In [ ]:
# FM
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.metrics import MeanSquaredError
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# 자료형 선언
tf.keras.backend.set_floatx('float32')

In [ ]:
X, Y = train.loc[:,['userID','location.locationId']], train.loc[:,'rating']

n = X.shape[0]
p = X.shape[1]
k = 10

In [ ]:
# FM 모델 선언  
class FM(tf.keras.Model):
    def __init__(self):
        super(FM, self).__init__()

        # 모델의 파라미터 정의
        self.w_0 = tf.Variable([0.0])
        self.w = tf.Variable(tf.zeros([p]))
        self.V = tf.Variable(tf.random.normal(shape=(p, k)))

    def call(self, inputs):
        linear_terms = tf.reduce_sum(tf.math.multiply(self.w, inputs), axis=1)

        interactions = 0.5 * tf.reduce_sum(
          tf.math.pow(tf.matmul(inputs, self.V), 2)
          - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.V, 2)),
          1,
          keepdims=False) 

        y_hat = tf.math.sigmoid(self.w_0 + linear_terms + interactions)

        return y_hat

In [ ]:
# 학습코드

# 배치 학습 역전파 함수 
def train_on_batch(model, optimizer, mse, inputs, targets):
    # 자동 미분
    with tf.GradientTape() as tape:
        y_pred = model(inputs) 
        loss = tf.keras.losses.mse(targets, y_pred)
    
    # loss를 모델의 파라미터로 편미분하여 gradients를 구한다.
    grads = tape.gradient(target=loss, sources=model.trainable_variables)

    # apply_gradients()를 통해 processed gradients를 적용한다.
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # mse: update할 때마다 정확도는 누적되어 계산된다.
    mse.update_state(targets, y_pred)

    return loss 

# 반복 학습 함수 
def train(epochs, lr, batch_size):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

    train_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_train, tf.float32), tf.cast(Y_train, tf.float32))).shuffle(500).batch(batch_size)

    test_ds = tf.data.Dataset.from_tensor_slices(
        (tf.cast(X_test, tf.float32), tf.cast(Y_test, tf.float32))).shuffle(200).batch(batch_size)

    model = FM()
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    mse = MeanSquaredError()
    loss_history = []

    for i in range(epochs):
      for x, y in train_ds:
          loss = train_on_batch(model, optimizer, mse, x, y)
          loss_history.append(loss)

      if i % 2== 0:
          print("스텝 {:03d}에서 누적 평균 손실: {:.4f}".format(i, np.mean(loss_history)))

    test_preds = []
    test_mse = MeanSquaredError()
    for x, y in test_ds:
        y_pred = model(x)
        # Converts tenser to array 
        test_preds.append((y.numpy(), y_pred.numpy())) 
        test_mse.update_state(y, y_pred)
    
    print("테스트 정확도: {:.4f}".format(test_mse.result().numpy()))

    return test_preds

# 모델 학습 후 예측 
def pred(df, epochs, lr, batch_size):
  model = train(epochs, lr, batch_size)
  preds = model(df.loc[:,['userID','location.locationId']])
  return preds

In [37]:
train(50, lr=0.0001, batch_size=20)

스텝 000에서 누적 평균 손실: 18.0886
스텝 002에서 누적 평균 손실: 18.0876
스텝 004에서 누적 평균 손실: 18.0877
스텝 006에서 누적 평균 손실: 18.0879
스텝 008에서 누적 평균 손실: 18.0879


KeyboardInterrupt: ignored

왤케 손시링 크게 나오냐 ㅜㅜ ㅜㅎㄹㅁ더ㅣㅜㅇ

# 3. 패키지를....

In [41]:
new_df

,userId,location.locationId,rating
0,0009720721B493BAE608AA592B9CE7AE,294197,0.0
1,00150C8ACD87B7A3A54497D923CB3358,294197,0.0
2,001608AFA00EC79804E14EE201018BBB,294197,0.0
3,00178BFBBC5B052A9E0845691F9585B4,294197,0.0
4,001A6C668BB5BE75A85603BBADAEBCD9,294197,0.0
...,...,...,...
82936017,FFD763E06A0001B0F8143C91D2EE35FB,20903295,0.0
82936018,FFF028A54917977AB9549FD75103ECDA,20903295,0.0
82936019,FFF66251C5DC520DA1F9D217419BD90A,20903295,0.0
82936020,FFFA706118AC068F63A24BD7452D4F80,20903295,0.0


In [43]:
test = new_df.query('rating == 0.0')
train = new_df.query('rating != 0.0')
print(test.shape, train.shape)

(82900038, 3) (35984, 3)


In [44]:
test

,userId,location.locationId,rating
0,0009720721B493BAE608AA592B9CE7AE,294197,0.0
1,00150C8ACD87B7A3A54497D923CB3358,294197,0.0
2,001608AFA00EC79804E14EE201018BBB,294197,0.0
3,00178BFBBC5B052A9E0845691F9585B4,294197,0.0
4,001A6C668BB5BE75A85603BBADAEBCD9,294197,0.0
...,...,...,...
82936017,FFD763E06A0001B0F8143C91D2EE35FB,20903295,0.0
82936018,FFF028A54917977AB9549FD75103ECDA,20903295,0.0
82936019,FFF66251C5DC520DA1F9D217419BD90A,20903295,0.0
82936020,FFFA706118AC068F63A24BD7452D4F80,20903295,0.0


In [45]:
train

,userId,location.locationId,rating
989,214B0FB3019C173168F2EBBF0077D671,294197,4.0
8964,2C7CEE044052FF0994EEE46ED9E216B3,299151,3.0
11029,72E2C54A84609D3FD2CFF9C29D26150E,299151,3.0
15705,11ABDC62AA61ED621186E341AE6A6356,299152,5.0
16215,214B0FB3019C173168F2EBBF0077D671,299152,3.0
...,...,...,...
82904839,E5F93B0B3B8683B98A7B3642B2BE2399,20403351,5.0
82912452,E5F93B0B3B8683B98A7B3642B2BE2399,20483591,5.0
82914172,214B0FB3019C173168F2EBBF0077D671,20494843,4.0
82927678,E5F93B0B3B8683B98A7B3642B2BE2399,20860800,5.0


In [47]:
# ! pip install git+https://github.com/coreylynch/pyFM

  Cloning https://github.com/coreylynch/pyFM to /tmp/pip-req-build-vuo8y8yr
  Running command git clone -q https://github.com/coreylynch/pyFM /tmp/pip-req-build-vuo8y8yr
  Created wheel for pyfm: filename=pyfm-0.0.0-cp36-cp36m-linux_x86_64.whl size=216897 sha256=a3bb5e14dd94fda6dfbaa5460d28200679819acc0910e055e6ee37525e6de51f
  Stored in directory: /tmp/pip-ephem-wheel-cache-r2o0sdu2/wheels/3b/d9/ef/1b148c527d39344632833679e79b3db1798a40b0f64f917b13
Successfully built pyfm


In [ ]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from pyfm import pylibfm

# Read in data

def loadData(filename,path=os.path.join("..","..","data")):
    data = []
    y = []
    users=set()
    items=set()
    with open(path+filename) as f:
        for line in f:
            (userId,locationId,rating)=line.split('\t')
            data.append({ "user_id": str(userId), "location_id": str(locationId)})
            y.append(float(rating))
            users.add(userId)
            items.add(locationId)

    return (data, np.array(y), users, items)

(train_data, y_train, train_users, train_items) = loadData("train_new.txt")
(test_data, y_test, test_users, test_items) = loadData("test_new.txt")
v = DictVectorizer() 
X_train = v.fit_transform(train_data) 
X_test = v.transform(test_data)

In [ ]:
from pyfm import pylibfm
# Build and train a Factorization Machine
fm = pylibfm.FM(num_factors=10, num_iter=100, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")

fm.fit(X_train,y_train) 

In [ ]:
# Evaluate 
preds = fm.predict(X_test) 
from sklearn.metrics import mean_squared_error
print("FM MSE: %.4f" % mean_squared_error(y_test,preds))

In [ ]:
test['pred'] = 0
for i in range(len(preds)):
  test[i]['pred'] = preds[i]

test